In [1]:
import pennylane as qml
# Devices for Eve and Bob with single-shot simulations
dev_eve = qml.device('default.qubit', wires=1, shots=1)
dev_bob = qml.device('default.qubit', wires=1, shots=1)
dev_no_eve = qml.device('default.qubit', wires=1, shots=1)

In [2]:
# Alice's encoding and Eve's measurement
@qml.qnode(dev_eve)
def alice_eve_circuit(bit, basis):
    if bit == 1:
        qml.PauliX(0)
    if basis == 'S':
        qml.Hadamard(0)
    return qml.sample(wires=0)

# Bob's measurement after Eve's interception
@qml.qnode(dev_bob)
def bob_circuit(eve_bit, bob_basis):
    if eve_bit == 1:
        qml.PauliX(0)
    if bob_basis == 'S':
        qml.Hadamard(0)
    return qml.sample(wires=0)

# Direct Alice to Bob communication (no Eve)
@qml.qnode(dev_no_eve)
def alice_bob_circuit(bit, alice_basis, bob_basis):
    if bit == 1:
        qml.PauliX(0)
    if alice_basis == 'S':
        qml.Hadamard(0)
    if bob_basis == 'S':
        qml.Hadamard(0)
    return qml.sample(wires=0)

In [ ]:
# Alice's initial setup
alice_bits = [0,1,1,0,0,0,1,0,1,1]
alice_choices = ['NS', 'NS', 'S', 'NS', 'S','NS', 'S', 'NS','NS', 'S']

print("Alice created the key of 10 bits:", alice_bits)
print("Alice's basis choices:", alice_choices)
print("Now, Alice 'sends' her qubits to Bob >>>>>>>>>>>>>>>>>>>>>>>>>>>>")


In [ ]:
print("***************** IF EVE INTERCEPTS! ******************")

# Eve intercepts and measures
eve_bits = []
for q in range(len(alice_bits)):
    eve_bit = alice_eve_circuit(alice_bits[q], alice_choices[q])
    eve_bits.append(int(eve_bit))

print("Eve measured bits:", eve_bits)
print("Superposition collapsed; Bob receives intercepted qubits.")

# Bob measures with his basis choices
bob_choices = ['S', 'NS', 'NS', 'NS', 'S','NS', 'NS', 'NS','S', 'S']
bob_bits = []
for q in range(len(eve_bits)):
    bob_bit = bob_circuit(eve_bits[q], bob_choices[q])
    bob_bits.append(int(bob_bit))

print("Bob's basis choices:", bob_choices)
print("Bob measured bits:", bob_bits)

# Derive effective keys where bases match
alice_key = [alice_bits[q] for q in range(10) if alice_choices[q] == bob_choices[q]]
bob_key = [bob_bits[q] for q in range(10) if alice_choices[q] == bob_choices[q]]

print("Effective Alice's key:", alice_key)
print("Effective Bob's key:", bob_key)

if alice_key == bob_key:
    print("SECURE! Keys match. Key ==", alice_key)
else:
    print("EVE DETECTED! Keys mismatch. Use another quantum channel.")

In [ ]:
print("\nWITHOUT EVE'S INTERCEPTION")
print("$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$")

# Bob measures directly without Eve's interference
bob_bits_no_eve = []
for q in range(len(alice_bits)):
    bob_bit = alice_bob_circuit(alice_bits[q], alice_choices[q], bob_choices[q])
    bob_bits_no_eve.append(int(bob_bit))

print("Bob's basis choices:", bob_choices)
print("Bob measured bits (no Eve):", bob_bits_no_eve)

# Derive effective keys where bases match
alice_key_ne = [alice_bits[q] for q in range(10) if alice_choices[q] == bob_choices[q]]
bob_key_ne = [bob_bits_no_eve[q] for q in range(10) if alice_choices[q] == bob_choices[q]]

print("Effective Alice's key (no Eve):", alice_key_ne)
print("Effective Bob's key (no Eve):", bob_key_ne)

print("||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||")
if alice_key_ne == bob_key_ne:
    print("SECURE! Keys match without Eve. KEY ==", alice_key_ne)
else:
    print("ERROR: Keys mismatch without Eve. Check simulation.")